# Init

In [80]:
!pip install torch
!pip install pytorch_pretrained_bert

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertForQuestionAnswering, BertForPreTraining ,BertAdam

# Bert Generate One By One

In [0]:
input_text = "[CLS] I go to school by bus [SEP] "
target_text = "我搭公車上學"

In [27]:
# Load pre-trained model tokenizer (vocabulary)
modelpath = "bert-base-chinese"
tokenizer = BertTokenizer.from_pretrained(modelpath)
model = BertForMaskedLM.from_pretrained(modelpath)

model.to('cuda')

example_pair = dict()

for i in range(0,len(target_text)+1):
  tokenized_text = tokenizer.tokenize(input_text)
  tokenized_text.extend(target_text[:i])
  tokenized_text.append('[MASK]')
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
#   for _ in range(512-len(indexed_tokens)):
#     indexed_tokens.append(0)
  tokens_tensor = torch.tensor([indexed_tokens]).to('cuda')
  
  loss_ids = [-1] * (len(tokenizer.convert_tokens_to_ids(tokenized_text))-1)
  if i == len(target_text):
    loss_ids.append(tokenizer.convert_tokens_to_ids(tokenizer.tokenize('[SEP]'))[0])
  else:
    loss_ids.append(tokenizer.convert_tokens_to_ids(target_text[i])[0])
#   for _ in range(512-len(loss_ids)):
#     loss_ids.append(-1)
  loss_tensors = torch.tensor([loss_ids]).to('cuda')
  
  example_pair[tokens_tensor] = loss_tensors
  print(tokenized_text,loss_ids,loss_ids[-1])
  print(len(indexed_tokens),len(loss_tensors))

['[CLS]', 'i', 'go', 'to', 'school', 'by', 'bus', '[SEP]', '[MASK]'] [-1, -1, -1, -1, -1, -1, -1, -1, 2769] 2769
9 1
['[CLS]', 'i', 'go', 'to', 'school', 'by', 'bus', '[SEP]', '我', '[MASK]'] [-1, -1, -1, -1, -1, -1, -1, -1, -1, 3022] 3022
10 1
['[CLS]', 'i', 'go', 'to', 'school', 'by', 'bus', '[SEP]', '我', '搭', '[MASK]'] [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1062] 1062
11 1
['[CLS]', 'i', 'go', 'to', 'school', 'by', 'bus', '[SEP]', '我', '搭', '公', '[MASK]'] [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 6722] 6722
12 1
['[CLS]', 'i', 'go', 'to', 'school', 'by', 'bus', '[SEP]', '我', '搭', '公', '車', '[MASK]'] [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 677] 677
13 1
['[CLS]', 'i', 'go', 'to', 'school', 'by', 'bus', '[SEP]', '我', '搭', '公', '車', '上', '[MASK]'] [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 2119] 2119
14 1
['[CLS]', 'i', 'go', 'to', 'school', 'by', 'bus', '[SEP]', '我', '搭', '公', '車', '上', '學', '[MASK]'] [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 102]

In [28]:
print(example_pair)

{tensor([[  101,   151,  8373,  8228,  9467,  8120, 10411,   102,   103]],
       device='cuda:0'): tensor([[  -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1, 2769]],
       device='cuda:0'), tensor([[  101,   151,  8373,  8228,  9467,  8120, 10411,   102,  2769,   103]],
       device='cuda:0'): tensor([[  -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1, 3022]],
       device='cuda:0'), tensor([[  101,   151,  8373,  8228,  9467,  8120, 10411,   102,  2769,  3022,
           103]], device='cuda:0'): tensor([[  -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1, 1062]],
       device='cuda:0'), tensor([[  101,   151,  8373,  8228,  9467,  8120, 10411,   102,  2769,  3022,
          1062,   103]], device='cuda:0'): tensor([[  -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1, 6722]],
       device='cuda:0'), tensor([[  101,   151,  8373,  8228,  9467,  8120, 10411,   102,  2769,  3022,
          1062,  6722,   103]], device='cuda:0'): tensor([[ -1,  -1,  -1,  -1,  -1

In [0]:
# Prepare optimizer
param_optimizer = list(model.named_parameters())

# hack to remove pooler, which is not used
# thus it produce None grad that break apex
param_optimizer = [n for n in param_optimizer if 'pooler' not in n[0]]

no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
optimizer = BertAdam(optimizer_grouped_parameters,
                             lr=5e-5,
                             warmup=0.1,
                             t_total=300)

In [39]:
model.train()
for i in range(0,60):
  eveloss = 0
  for k,v in example_pair.items():
    loss = model(k,masked_lm_labels=v)
    eveloss += loss.mean().item()
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
  print("step "+ str(i) + " : " + str(eveloss))

step 0 : 37.36085796356201
step 1 : 23.70779514312744
step 2 : 21.444865226745605
step 3 : 18.553422927856445
step 4 : 17.417753219604492
step 5 : 16.897245407104492
step 6 : 15.978704452514648
step 7 : 17.163260459899902
step 8 : 16.674589157104492
step 9 : 16.660064697265625
step 10 : 16.22218418121338
step 11 : 16.475232124328613
step 12 : 15.244463920593262
step 13 : 14.150313377380371
step 14 : 16.29252815246582
step 15 : 16.90859889984131
step 16 : 15.194635391235352
step 17 : 17.376182556152344
step 18 : 16.083887100219727
step 19 : 16.41118049621582
step 20 : 15.8125
step 21 : 14.576831817626953
step 22 : 13.978545188903809
step 23 : 16.20425033569336
step 24 : 15.351747512817383


KeyboardInterrupt: ignored

In [37]:
model.eval()
for k,v in example_pair.items():
    predictions = model(k)
    predicted_index = torch.argmax(predictions[0,len(predictions[0])-1]).item()
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])
    if '[SEP]' in predicted_token:
      break
    print(predicted_token)


['[PAD]']
['[PAD]']
['[PAD]']
['[PAD]']
['[PAD]']
['[PAD]']
['[PAD]']


# Bert Generate One Time



In [0]:
input_text = "[CLS] I go to school by bus [SEP] "
target_text = "我搭公車上學"

In [0]:
# Load pre-trained model tokenizer (vocabulary)
modelpath = "bert-base-chinese"
tokenizer = BertTokenizer.from_pretrained(modelpath)
model = BertForMaskedLM.from_pretrained(modelpath)
model.to('cuda')

tokenized_text = tokenizer.tokenize(input_text)
for i in target_text:
  tokenized_text.append('[MASK]')
# tokenized_text.append('[SEP]')
for _ in range(128-len(tokenized_text)):
  tokenized_text.append('[MASK]')
# tokenized_text.append('[MASK]')
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

tokens_tensor = torch.tensor([indexed_tokens]).to('cuda')

loss_ids = []
loss_ids = [-1] * (len(tokenizer.tokenize(input_text)))
# loss_ids.extend(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(input_text)))
for i in target_text:
  loss_ids.append(tokenizer.convert_tokens_to_ids(i)[0])
loss_ids.append(tokenizer.convert_tokens_to_ids(['[SEP]'])[0])
for _ in range(128-len(loss_ids)):
  loss_ids.append(-1)
loss_tensors = torch.tensor([loss_ids]).to('cuda')

In [208]:
print(tokens_tensor,loss_tensors)
print(tokenizer.convert_ids_to_tokens(indexed_tokens))

tensor([[  101,   151,  8373,  8228,  9467,  8120, 10411,   102,   103,   103,
           103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
           103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
           103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
           103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
           103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
           103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
           103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
           103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
           103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
           103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
           103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
           103,   103,   103,   103,   103,   103,  

In [0]:
# # Prepare optimizer
# param_optimizer = list(model.named_parameters())

# # hack to remove pooler, which is not used
# # thus it produce None grad that break apex
# param_optimizer = [n for n in param_optimizer if 'pooler' not in n[0]]

# no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
# optimizer_grouped_parameters = [
#         {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.05},
#         {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.01}
#         ]
# optimizer = BertAdam(optimizer_grouped_parameters,
#                              lr=5e-5)

# optimizer = torch.optim.Adam(model.parameters(), lr=5e-7)
# optimizer = torch.optim.SGD(model.parameters(), lr = 5e-5, momentum=0.9)
optimizer = torch.optim.Adamax(model.parameters(), lr = 5e-5)

In [212]:
model.train()
for i in range(0,300):
  loss = model(tokens_tensor,masked_lm_labels=loss_tensors)
  eveloss = loss.mean().item()
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  print("step "+ str(i) + " : " + str(eveloss))

step 0 : 8.128867149353027
step 1 : 6.8241424560546875
step 2 : 5.619769096374512
step 3 : 5.153357982635498
step 4 : 5.289130210876465
step 5 : 4.376535892486572
step 6 : 4.193200588226318
step 7 : 4.259396553039551
step 8 : 4.070324420928955
step 9 : 3.9747025966644287
step 10 : 4.1999101638793945
step 11 : 3.933626413345337
step 12 : 3.8321549892425537
step 13 : 3.891087055206299
step 14 : 3.803987741470337
step 15 : 3.8564608097076416
step 16 : 3.6616151332855225
step 17 : 3.6182186603546143
step 18 : 3.579364776611328
step 19 : 3.6259236335754395
step 20 : 3.615184783935547
step 21 : 3.3506252765655518
step 22 : 3.3550331592559814
step 23 : 3.4384877681732178
step 24 : 3.405526876449585
step 25 : 3.380850315093994
step 26 : 3.18172287940979
step 27 : 3.339765787124634
step 28 : 3.053271770477295
step 29 : 3.0905978679656982
step 30 : 2.9776012897491455
step 31 : 2.66851544380188
step 32 : 3.1134982109069824
step 33 : 2.7148592472076416
step 34 : 2.677067995071411
step 35 : 3.07570

In [0]:
model.eval()
with torch.no_grad():
  predictions = model(tokens_tensor)
  start = len(tokenizer.tokenize(input_text))
  while start < len(predictions[0]):
    predicted_index = torch.argmax(predictions[0,start]).item()
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])
    if '[SEP]' in predicted_token:
        break
    print(predicted_token)
    start+=1

# Bert Generate LSTM




In [0]:
input_text = "[CLS] I go to school by bus [SEP] "
target_text = "我搭公車上學"

In [0]:
# Load pre-trained model tokenizer (vocabulary)
modelpath = "bert-base-chinese"
tokenizer = BertTokenizer.from_pretrained(modelpath)
example_pair = dict()

for i in range(0,len(target_text)+1):
  tokenized_text = tokenizer.tokenize(input_text)
  tokenized_text.extend(target_text[:i])
  tokenized_text.append('[MASK]')
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  tokens_tensor = torch.tensor([indexed_tokens]).to('cuda')
  
  loss_ids = [-1] * (len(tokenizer.convert_tokens_to_ids(tokenized_text))-1)
  if i == len(target_text):
    loss_ids.append(tokenizer.convert_tokens_to_ids(tokenizer.tokenize('[SEP]'))[0])
  else:
    loss_ids.append(tokenizer.convert_tokens_to_ids(target_text[i])[0])
  loss_tensors = torch.tensor([loss_ids]).to('cuda')
  
  example_pair[tokens_tensor] = loss_tensors

In [0]:
class BertLSTM(nn.Module):
    def __init__(self):
        super().__init__()
        modelpath = "bert-base-chinese"
        self.bert = BertModel.from_pretrained(modelpath)
        self.rnn = nn.LSTM(num_layers=2,dropout=0.2, input_size=768, hidden_size=768//2)
        self.fc = nn.Linear(384, self.bert.config.vocab_size)
        
    def forward(self, x,y=None):
        
        self.bert.train()
        encoded_layers, _ = self.bert(x)
        for i in encoded_layers:
          enc, _ = self.rnn(i)
        logits = self.fc(enc)
        if y is not None:
          loss_fct  = nn.CrossEntropyLoss(ignore_index=-1)
          loss = loss_fct(logits.view(-1, logits.shape[-1]), y.view(-1))
          return loss
        return logits

In [0]:
model = BertLSTM()
optimizer = torch.optim.Adam(model.parameters(), lr = 5e-5)

In [79]:
model.to('cuda')
model.train()
for i in range(0,115):
  eveloss = 0
  optimizer.zero_grad()
  for k,v in example_pair.items():
    loss = model(k,v)
    eveloss += loss.mean().item()
    loss.backward()
    optimizer.step()
  print("step "+ str(i) + " : " + str(eveloss))

step 0 : 15.10863208770752
step 1 : 15.151559352874756
step 2 : 15.35487985610962
step 3 : 15.099365234375
step 4 : 14.601593494415283
step 5 : 15.178351879119873
step 6 : 15.133653163909912
step 7 : 15.380486011505127
step 8 : 14.865607738494873


KeyboardInterrupt: ignored

In [78]:
model.eval()
for k,v in example_pair.items():
    predictions = model(k)
    predicted_index = torch.argmax(predictions[0,-1]).item()
    if predicted_index < model.bert.config.vocab_size:
      predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])
    print(predicted_token)

['我']
['我']
['我']
['我']
['我']
['我']
['我']
